In [1]:
"""
To retrieve the public cell typing training data
"""

'\nTo retrieve the public cell typing training data\n'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from caveclient import CAVEclient
client = CAVEclient('minnie65_phase3_v1') # doesn't work just by itself (all commands hang)

In [4]:
client.materialize.get_tables()

['allen_v1_column_thalamic',
 'baylor_gnn_cell_type_fine_model_v2',
 'apl_functional_coreg',
 'nucleus_alternative_points',
 'allen_column_mtypes_v2',
 'column_atype',
 'connectivity_groups_v507',
 'new_table_test',
 'connectivity_groups_v795',
 'proofreading_functional_coreg',
 'bodor_pt_cells',
 'proofreading_apl_multisoma',
 'aibs_metamodel_mtypes_v661_v2',
 'aibs_metamodel_celltypes_v661_corrections',
 'allen_column_l5it_types',
 'allen_v1_column_types_slanted',
 'baylor_e_i_model_v1',
 'allen_column_mtypes_v1',
 'allen_v1_column_types_slanted_ref',
 'test_nvq_table',
 'aibs_column_nonneuronal_ref',
 'nucleus_ref_neuron_svm',
 'apl_functional_coreg_v2',
 'validated_opc_synapses',
 'coregistration_manual',
 'aibs_soma_nuc_exc_mtype_preds_v117',
 'apl_functional_coreg_vess_fwd',
 'test_nvq_table_status',
 'vortex_compartment_targets',
 'baylor_log_reg_cell_type_coarse_v1',
 'test_synapse_target_proofreading',
 'test_synapse_target_broadcast',
 'l23exc_column',
 'functional_properties

In [9]:
public_cave_ground_truth_cell_types = "allen_v1_column_types_slanted_ref"
public_nucleus_table = "nucleus_detection_v0"

In [14]:
from datasci_tools import pandas_utils as pu

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [15]:
nucleus_db = client.materialize.query_table(public_nucleus_table)

In [16]:
pu.rename_columns??

Signature: pu.rename_columns(df, columns_name_map, in_place=False)
Docstring: <no docstring>
Source:   
def rename_columns(df,columns_name_map,in_place = False,):
    return_value = df.rename(columns=columns_name_map,inplace = in_place)
    if in_place:
        return df
    else:
        return return_value
File:      /neurd_packages/datasci_tools/datasci_tools/pandas_utils.py
Type:      function

In [21]:
nucleus_df = nucleus_db[["id","pt_root_id"]]
nucleus_df = pu.rename_columns(nucleus_df,{"id":"nucleus_id",})
nucleus_df

,nucleus_id,pt_root_id
0,730537,0
1,373879,864691136090135607
2,601340,0
3,201858,864691135373893678
4,600774,0
...,...,...
144115,598753,864691135743752909
144116,111162,864691134912248365
144117,795176,0
144118,528334,864691135968943973


In [22]:
df = client.materialize.query_table(public_cave_ground_truth_cell_types)
cell_type_df = df[["cell_type","pt_root_id"]]
cell_type_df

,cell_type,pt_root_id
0,23P,864691136021936376
1,6P-CT,864691136487559186
2,23P,864691135784109363
3,23P,864691135694415551
4,BPC,864691135759892302
...,...,...
1352,5P-PT,864691135503182685
1353,5P-PT,864691136335150259
1354,5P-PT,864691135688186080
1355,5P-PT,864691135503435613


In [28]:
import pandas as pd
nuc_cell_type_df = pd.merge(cell_type_df,nucleus_df,how="inner",on=["pt_root_id"])
nuc_cell_type_df = nuc_cell_type_df.rename(columns=dict(pt_root_id='segment_id'),inplace = False)
nuc_cell_type_df = nuc_cell_type_df[['segment_id','nucleus_id','cell_type']]
nuc_cell_type_df

,segment_id,nucleus_id,cell_type
0,864691136021936376,258319,23P
1,864691136487559186,276438,6P-CT
2,864691135784109363,260552,23P
3,864691135694415551,260263,23P
4,864691135759892302,262898,BPC
...,...,...,...
1352,864691135503182685,267029,5P-PT
1353,864691136335150259,267294,5P-PT
1354,864691135688186080,303145,5P-PT
1355,864691135503435613,302955,5P-PT


In [29]:
pu.df_to_csv(nuc_cell_type_df,"public_cave_ground_truth_cell_types_with_nucleus.csv")

'/neurd_packages/NEURD/Applications/Tutorials/GNN_Cell_Typing/training_data/public_cave_ground_truth_cell_types_with_nucleus.csv'